# H2O.ai

In [ ]:
import sys
lib_dir = "../../libs/"
if lib_dir not in sys.path:
    sys.path.append(lib_dir)

In [ ]:
import automl

import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.automl import H2OAutoML

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
#h2o.cluster().shutdown()

In [ ]:
h2o.init(min_mem_size=6)

In [ ]:
df = pd.read_pickle('../../data/Visual_well_being/merged_features.pickle')

In [ ]:
df.head()

In [ ]:
df.info(verbose=True)

In [ ]:
target_features = ['P', 'E', 'R', 'M', 'A', 'PERMA']

for feature in target_features:
    
    print('### Runnning AutoML for Predicting {} ###'.format(feature))
    
    aux_target_features = target_features.copy()
    
    #Remove remaining features as they could leak info into our dataset
    
    aux_target_features.remove(feature)
    
    df_pred = df.drop(aux_target_features, axis=1).reset_index()
    
    df_train = df_pred.sample(frac=0.7)
    
    h2o_train = h2o.H2OFrame(df_train)
    h2o_test = h2o.H2OFrame(df_pred[~np.isin(df_pred.user_id, df_train.user_id)])
    
    exclude_algos = ['DeepLearning']
    
    automl.generate_predictions_automl(h2o_train, h2o_test, exclude_algos, 5, 123, 10, feature)

In [ ]:
df.loc[:,target_features].describe()